In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.2)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.65 sec - Loss 0.097695 - ACC 79.44% - ACC Mean 79.44% - AUC 84.53% - AUC Mean 84.53% - Deter 000
Ite 00050 - 0.10 sec - Loss 0.091798 - ACC 79.70% - ACC Mean 79.36% - AUC 84.63% - AUC Mean 84.45% - Deter 043
Ite 00100 - 0.13 sec - Loss 0.076857 - ACC 80.56% - ACC Mean 79.98% - AUC 85.14% - AUC Mean 84.76% - Deter 093
Ite 00150 - 0.45 sec - Loss 0.055789 - ACC 78.37% - ACC Mean 79.85% - AUC 85.14% - AUC Mean 85.49% - Deter 019
Ite 00200 - 0.12 sec - Loss 0.046559 - ACC 78.88% - ACC Mean 78.72% - AUC 85.33% - AUC Mean 85.44% - Deter 069
Ite 00250 - 0.09 sec - Loss 0.044456 - ACC 79.63% - ACC Mean 79.34% - AUC 85.71% - AUC Mean 85.74% - Deter 119
Ite 00300 - 0.11 sec - Loss 0.043830 - ACC 80.09% - ACC Mean 80.11% - AUC 85.89% - AUC Mean 86.07% - Deter 019
Ite 00350 - 0.14 sec - Loss 0.043458 - ACC 78.92% - ACC Mean 80.12% - AUC 85.65% - AUC Mean 86.07% - Deter 069
Ite 00400 - 0.44 sec - Loss 0.043164 - ACC 80.44% - ACC Mean 80.02% - AUC 86.27% - AUC Mean 86.01% - Deter 119
I

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 76.17% +- 0.0% - auc: 83.6% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 76.6% +- 2.99% - auc: 83.06% +- 2.5%
SFIL RUN 2/10 - acc: 75.11% +- 2.79% - auc: 80.54% +- 2.8%
SFIL RUN 3/10 - acc: 72.8% +- 2.42% - auc: 76.88% +- 2.5%
SFIL RUN 4/10 - acc: 75.63% +- 2.47% - auc: 79.62% +- 1.64%
SFIL RUN 5/10 - acc: 75.45% +- 6.0% - auc: 80.37% +- 4.28%
SFIL RUN 6/10 - acc: 75.09% +- 5.77% - auc: 77.8% +- 5.59%
SFIL RUN 7/10 - acc: 74.44% +- 2.26% - auc: 78.84% +- 2.58%
SFIL RUN 8/10 - acc: 78.75% +- 1.95% - auc: 81.91% +- 2.44%
SFIL RUN 9/10 - acc: 74.93% +- 4.87% - auc: 81.31% +- 2.29%
SFIL RUN 10/10 - acc: 73.29% +- 7.25% - auc: 78.81% +- 8.19%
SFIL GLOBAL - acc: 75.21% +- 1.58% - auc: 79.91% +- 1.8%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 78.71% +- 2.29% - auc: 84.63% +- 1.42%
PFIL RUN 2/10 - acc: 80.72% +- 5.99% - auc: 84.56% +- 6.04%
PFIL RUN 3/10 - acc: 76.53% +- 6.18% - auc: 82.01% +- 2.89%
PFIL RUN 4/10 - acc: 77.56% +- 7.02% - auc: 84.51% +- 5.91%
PFIL RUN 5/10 - acc: 81.4% +- 2.22% - auc: 86.07% +- 2.09%
PFIL RUN 6/10 - acc: 76.02% +- 3.89% - auc: 82.49% +- 3.09%
PFIL RUN 7/10 - acc: 79.75% +- 3.23% - auc: 85.35% +- 2.35%
PFIL RUN 8/10 - acc: 80.55% +- 4.56% - auc: 85.74% +- 4.39%
PFIL RUN 9/10 - acc: 74.58% +- 6.79% - auc: 78.85% +- 4.5%
PFIL RUN 10/10 - acc: 78.25% +- 2.86% - auc: 83.51% +- 3.95%
PFIL GLOBAL - acc: 78.41% +- 2.13% - auc: 83.77% +- 2.07%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 78.17% +- 4.3% - auc: 83.31% +- 2.67%
SPOL RUN 2/10 - acc: 77.09% +- 5.19% - auc: 82.44% +- 3.88%
SPOL RUN 3/10 - acc: 76.01% +- 3.13% - auc: 80.27% +- 3.2%
SPOL RUN 4/10 - acc: 75.37% +- 3.28% - auc: 81.29% +- 4.24%
SPOL RUN 5/10 - acc: 80.19% +- 3.84% - auc: 86.71% +- 2.85%
SPOL RUN 6/10 - acc: 76.94% +- 5.3% - auc: 82.56% +- 4.72%
SPOL RUN 7/10 - acc: 75.99% +- 3.87% - auc: 81.85% +- 3.93%
SPOL RUN 8/10 - acc: 78.17% +- 2.35% - auc: 81.99% +- 2.38%
SPOL RUN 9/10 - acc: 74.25% +- 6.48% - auc: 80.86% +- 4.31%
SPOL RUN 10/10 - acc: 76.3% +- 4.35% - auc: 81.1% +- 3.27%
SPOL GLOBAL - acc: 76.85% +- 1.59% - auc: 82.24% +- 1.71%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 77.67% +- 3.79% - auc: 82.61% +- 3.24%
PPOL RUN 2/10 - acc: 78.57% +- 6.29% - auc: 83.31% +- 6.19%
PPOL RUN 3/10 - acc: 77.54% +- 2.59% - auc: 83.09% +- 2.49%
PPOL RUN 4/10 - acc: 78.35% +- 4.77% - auc: 84.35% +- 3.84%
PPOL RUN 5/10 - acc: 77.46% +- 5.13% - auc: 83.18% +- 4.46%
PPOL RUN 6/10 - acc: 75.12% +- 3.93% - auc: 81.6% +- 2.34%
PPOL RUN 7/10 - acc: 75.05% +- 3.06% - auc: 83.47% +- 4.53%
PPOL RUN 8/10 - acc: 77.43% +- 5.21% - auc: 81.74% +- 6.04%
PPOL RUN 9/10 - acc: 73.79% +- 6.21% - auc: 81.14% +- 3.79%
PPOL RUN 10/10 - acc: 76.75% +- 2.94% - auc: 84.4% +- 3.76%
PPOL GLOBAL - acc: 76.77% +- 1.5% - auc: 82.89% +- 1.06%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is even with PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
